# ステップ3：Snowflakeでクラウドストレージ統合を作成する

**`CREATE STORAGE INTEGRATION`** コマンドを使用して、SnowflakeとS3クラウドストレージ間の連携を定義するストレージ統合を作成します。

### ストレージ統合とは？

ストレージ統合は、以下の情報を格納するSnowflakeオブジェクトです。

* **IAMユーザー:** SnowflakeがS3クラウドストレージ用に自動生成したIDおよびアクセス管理（IAM）ユーザー。
* **ストレージロケーション:** 許可またはブロックするストレージの場所（S3バケットなど）のリスト。

### 主な利点

* **認証情報の省略:** クラウドプロバイダーの管理者が、Snowflakeによって生成されたIAMユーザーにストレージロケーションへのアクセス許可を付与します。これにより、ユーザーはステージ作成時やデータロード時に認証情報（APIキーなど）を提供する手間を省けます。
* **複数ステージのサポート:** 単一のストレージ統合で、複数の外部（S3）ステージをサポートできます。ただし、各ステージ定義のURLは、統合の **`STORAGE_ALLOWED_LOCATIONS`** パラメーターで指定されたS3バケット（およびオプションのパス）と一致している必要があります。

> **注釈**
>
> このSQLコマンドを実行できるのは、アカウント管理者（**`ACCOUNTADMIN`** ロールを持つユーザー）またはグローバル **`CREATE INTEGRATION`** 権限を持つロールのみです。

In [ ]:
CREATE or replace STORAGE INTEGRATION aaaaaaaa_s3_integration
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = 'S3'
  ENABLED = TRUE
  STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::021891591907:role/snowflakerole_aaaaaaaa_hwatari'
  STORAGE_ALLOWED_LOCATIONS = ('s3://aaaaaaaa-hwatari/')

# ステップ4：Snowflakeアカウントの AWS IAM ユーザーを取得する
Snowflakeアカウント用に自動的に作成された IAM ユーザーの ARN を取得するには、 DESCRIBE INTEGRATION を使用します。

In [ ]:
DESC INTEGRATION aaaaaaaa_s3_integration;

## ステップ5：バケットオブジェクトにアクセスするために IAM ユーザー権限を付与する

次の手順では、S3バケットを使用してデータをロードおよびアンロードできるように、 AWS 管理コンソールでSnowflakeの IAM アクセス許可を構成する方法を説明します。

1.  AWS 管理コンソールにログインします。
2.  `IAM` を選択します。
3.  左側のナビゲーションペインから `Roles` を選択します。
4.  「ステップ2: IAM AWS ロールを作成する」で作成したロールを選択します。
5.  `Trust relationships` タブを選択します。
6.  `Edit trust policy` を選択します。
7.  「ステップ4：Snowflakeアカウントの AWS IAM ユーザーを取得する」で記録した `DESC STORAGE INTEGRATION` の出力値でポリシードキュメントを変更します。

### IAM ロールのポリシードキュメント

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::392051336358:user/3r201000-s"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "sts:ExternalId": "HU05376_SFCRole=7_l0cW1Ki71pU8t9bVpy2QZnPclME="
                }
            }
        }
    ]
}
```

# ステップ6: 外部ステージを作成する¶
ステップ3：Snowflakeでクラウドストレージ統合を作成する （このトピック内）で作成したストレージ統合を参照する、外部（つまり、S3）ステージを作成します。

In [ ]:
USE SCHEMA aaaaaaaa.public;

CREATE or replace STAGE aaaaaaaa_s3_stage
  STORAGE_INTEGRATION = aaaaaaaa_s3_integration
  URL = 's3://aaaaaaaa-hwatari/'